In [125]:
import requests
import pandas as pd
pd.options.display.max_columns = None

In [112]:
df = pd.read_csv('./data/occurrence.txt', sep='\t')

/home/davidgp/anaconda3/envs/malif/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (198,237) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [140]:
df = df[df['kingdom'] == 'Animalia']
df_order = df[['order']].drop_duplicates().dropna()
df_family = df[['family']].drop_duplicates().dropna()
df_genus = df[['genus']].drop_duplicates().dropna()

In [143]:
df_order.head()

,order
0,Tetraodontiformes
1,Perciformes
2,Anguilliformes
4,Nudibranchia
5,Carcharhiniformes


In [144]:
df_family.head()

,family
0,Ostraciidae
1,Pomacentridae
2,Muraenidae
4,Chromodorididae
5,Carcharhinidae


In [146]:
df_genus.head()

,genus
0,Ostracion
1,Stegastes
2,Pseudechidna
3,Gymnothorax
4,Chromodoris


In [116]:
def get_id(taxon):
    query = "https://eol.org/api/search/1.0.json?q=" + taxon
    request = requests.get(query)
    results = request.json()['results']
    for i in range(len(results)):
        if results[i]['title'] == taxon:
            return int(results[i]['id'])

In [154]:
df_order['id_order_eol'] = df_order['order'].apply(get_id)

In [117]:
df_family['id_family_eol'] = df_family['family'].apply(get_id)

In [149]:
df_genus['id_genus_eol'] = df_genus['genus'].apply(get_id)

In [155]:
df_order['id_order_eol'].isna().sum()

0

In [156]:
df_order[df_order['id_order_eol'].isna()]

,order,id_order_eol


In [127]:
df_family['id_family_eol'].isna().sum()

1

In [123]:
df_family[df_family['id_family_eol'].isna()]

,family,id_family_eol
18440,Cassidae,NaN


In [153]:
df_family.dropna(inplace=True)
df_family.shape

(639, 1)

In [150]:
df_genus['id_genus_eol'].isna().sum()

4

In [151]:
df_genus[df_genus['id_genus_eol'].isna()]

,genus,id_genus_eol
127,Periclimenes,NaN
3051,Orconectes,NaN
11659,Labeo,NaN
31025,Plesionika,NaN


In [152]:
df_genus.dropna(inplace=True)
df_genus.shape

(1691, 2)

In [133]:
def get_common_name(id_eol):
    query = "https://eol.org/api/pages/1.0/" + str(id_eol)+ ".json?common_names=true"
    request = requests.get(query)
    
    vernacular_names = request.json()['taxonConcept']['vernacularNames']
    
    for i in range(len(vernacular_names)):
        language = vernacular_names[i]['language']
        eol_preferred = vernacular_names[i]['eol_preferred']
        if language == 'en' and eol_preferred == True:
            return vernacular_names[i]['vernacularName']
        

In [134]:
df_family['family_common_name'] = df_family['id_family_eol'].apply(get_common_name)

<ipython-input-134-49684335588c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_family['family_common_name'] = df_family['id_family_eol'].apply(get_common_name)


In [135]:
df_family[df_family['family_common_name'].isna()]

,family,id_family_eol,family_common_name
4,Chromodorididae,46449475,None
11,Myrrhinidae,57281379,None
12,Serrasalmidae,47066240,None
18,Octopodidae,2322,None
44,Phyllidiidae,46450227,None
...,...,...,...
35774,Sertularellidae,51259906,None
36074,Prochilodontidae,6898,None
36312,Colossendeidae,6266,None
37537,Calliostomatidae,2376,None


In [136]:
get_common_name(46449475)

In [105]:
query = "https://eol.org/api/pages/1.0/" + str(6898)+ ".json?common_names=true"
request = requests.get(query)
for i in range(len(request.json()['taxonConcept']['vernacularNames'])):
    language = request.json()['taxonConcept']['vernacularNames'][i]['language']
    eol_preferred = request.json()['taxonConcept']['vernacularNames'][i]['eol_preferred']
    print(request.json()['taxonConcept']['vernacularNames'][i])
    #if language == 'en' and eol_preferred == True:
        #print(request.json()['taxonConcept']['vernacularNames'][i]['vernacularName'])

{'vernacularName': 'Barbensalmler', 'language': 'de', 'eol_preferred': True}
{'vernacularName': 'Nachtzalmen', 'language': 'nl', 'eol_preferred': True}
{'vernacularName': 'Paprastieji tykotojai', 'language': 'lt', 'eol_preferred': True}
{'vernacularName': 'Proquilodòntid', 'language': 'ca', 'eol_preferred': True}
{'vernacularName': 'Прохилодонтовые', 'language': 'ru', 'eol_preferred': True}
{'vernacularName': 'усести пастрмкопрапови', 'language': 'mk', 'eol_preferred': True}
{'vernacularName': 'کاراسین\u200cهای قیف\u200cدهان', 'language': 'fa', 'eol_preferred': True}
{'vernacularName': 'วงศ์ปลาอินซีเน็ต', 'language': 'th', 'eol_preferred': True}
{'vernacularName': 'プロキロダス科', 'language': 'jp', 'eol_preferred': True}
{'vernacularName': 'Barbensalmler', 'language': 'de', 'eol_preferred': False}
{'vernacularName': 'Nachtzalmen', 'language': 'nl', 'eol_preferred': False}
{'vernacularName': 'Paprastieji tykotojai', 'language': 'lt', 'eol_preferred': False}
{'vernacularName': 'Proquilodòntid'